<a href="https://colab.research.google.com/github/CargoCultScientist/markov-ball/blob/main/markov_random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def build_transition_matrix():
    """
    Constructs the transition matrix for a 4-player Cyberball game with random passing.
    Each state is (holder, k), where:
      - holder in {0, 1, 2, 3} = {P1, P2, P3, T}
      - k in {0, 1, 2, 3} is how many consecutive throws T has waited (capped at 3).
    Returns:
      states: a list of all 16 states (holder, k) in a fixed order
      P: the 16x16 transition matrix (rows sum to 1)
    """
    # Define the four players; let index 0 be focal player P1, index 3 be target T
    players = [0, 1, 2, 3]

    # All possible states: 4 holders x 4 "k-values"
    states = [(h, k) for h in players for k in range(4)]
    n_states = len(states)  # 16

    # Helper to map a state (h, k) -> index in the matrix
    state_index = {st: i for i, st in enumerate(states)}

    # Initialize transition matrix (16 x 16) with zeros
    P = np.zeros((n_states, n_states))

    # Populate the matrix
    for s_idx, (holder, k) in enumerate(states):
        # holder can pass to 3 other players, each with probability 1/3
        for recipient in players:
            if recipient != holder:
                # Probability of going from 'holder' to 'recipient'
                prob = 1.0 / 3.0

                # Determine next k-value based on whether T receives the ball
                if recipient == 3:  # passing to T resets the "due" count to 0
                    k_next = 0
                else:
                    # If T does not get the ball, increment k up to 3
                    k_next = min(k + 1, 3)

                # The next state is (recipient, k_next)
                next_idx = state_index[(recipient, k_next)]
                P[s_idx, next_idx] += prob

    return states, P

def compute_stationary_distribution(P, max_iter=10000, tol=1e-12):
    """
    Computes the stationary distribution of a transition matrix P using
    iterative power-method-like updates: pi_{t+1} = pi_t * P.
    Args:
      P: np.array of shape (16,16), the transition matrix
      max_iter: maximum number of iterations
      tol: convergence tolerance
    Returns:
      pi: 1D np.array of length 16, representing the stationary distribution
    """
    n_states = P.shape[0]
    # Start with a uniform distribution
    pi = np.ones(n_states) / n_states

    for _ in range(max_iter):
        pi_next = pi.dot(P)
        # Check for convergence
        if np.linalg.norm(pi_next - pi) < tol:
            return pi_next
        pi = pi_next
    return pi  # Return whatever we have if not converged, though it should converge.

def compute_transgression_ratio(states, P, pi):
    """
    Computes the long-run Transgression Ratio (TR) for the focal player (P1=0)
    given the stationary distribution pi. A transgression occurs when:
      - The focal player P1 (holder=0) has the ball
      - T (index=3) is 'due' -> k=3
      - P1 passes to a player that is not T.
    Because each pass is random among the other 3 players, the probability of
    skipping T is 2/3 in that scenario.
    TR = fraction of P1 throws that skip T when T is due, in the long run.
    """
    # Identify states of interest:
    # (P1, k) for k in 0..3 => the fraction of time P1 holds the ball
    # (P1, 3) => T is due while P1 is holder
    # The index for (P1,3):
    idx_p1_k3 = states.index((0, 3))

    # Probability that we are in (P1,3) in steady-state
    prob_p1_k3 = pi[idx_p1_k3]

    # Probability that we are in (P1,k) for any k => fraction of time P1 has ball
    prob_p1_any = sum(pi[states.index((0, k))] for k in range(4))

    # In (P1,3), a transgression occurs with probability 2/3 (choosing someone besides T)
    # so fraction of all P1 throws that are transgressions = (prob_p1_k3 * 2/3) / prob_p1_any.
    if prob_p1_any < 1e-15:
        return 0.0  # degenerate case, should not happen in a fair 4-player chain
    return (prob_p1_k3 * (2.0/3.0)) / prob_p1_any

def main():
    states, P = build_transition_matrix()
    pi = compute_stationary_distribution(P)
    TR = compute_transgression_ratio(states, P, pi)

    # Print results
    print("States (index -> (holder, k)):")
    for i, s in enumerate(states):
        print(i, "->", s)
    print("\nStationary distribution (sums to ~1):\n", pi)
    print("\nLong-run transgression ratio for P1:", TR)

if __name__ == "__main__":
    main()


States (index -> (holder, k)):
0 -> (0, 0)
1 -> (0, 1)
2 -> (0, 2)
3 -> (0, 3)
4 -> (1, 0)
5 -> (1, 1)
6 -> (1, 2)
7 -> (1, 3)
8 -> (2, 0)
9 -> (2, 1)
10 -> (2, 2)
11 -> (2, 3)
12 -> (3, 0)
13 -> (3, 1)
14 -> (3, 2)
15 -> (3, 3)

Stationary distribution (sums to ~1):
 [0.         0.08333333 0.05555556 0.11111111 0.         0.08333333
 0.05555556 0.11111111 0.         0.08333333 0.05555556 0.11111111
 0.25       0.         0.         0.        ]

Long-run transgression ratio for P1: 0.2962962962962963
